In [0]:
import json
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.cross_validation import train_test_split

In [0]:
from google.colab import files
uploaded = files.upload()



In [190]:
df = pd.read_csv("bciData.csv")

label=df.pop("label") # pop off labels to new group
print(df.shape)



print(df.head())


(4198, 4)
         a        b        c        d
0  1581.21  2062.57   608.05   334.41
1  1550.65  1990.27  1340.54  1082.14
2  1578.14  2034.80   598.77   323.75
3  1556.88  2012.28  1008.58   717.68
4  1561.22  2012.21  1003.91   715.37


In [0]:
df=df.values


In [192]:
train_labels = pd.get_dummies(label)

# make np array
train_labels = train_labels.values
print(train_labels.shape)

# print y_test

x_train,x_test,y_train,y_test = train_test_split(df,train_labels,test_size=0.2)
# so now we have predictors and y values, separated into test and train
print len(x_train)
print len(x_test)

# print y_test

x_train,x_test,y_train,y_test = np.array(x_train,dtype='float32'), np.array(x_test,dtype='float32'),np.array(y_train,dtype='float32'),np.array(y_test,dtype='float32')

(4198, 2)
3358
840


In [0]:
x = tf.placeholder(tf.float32, [None, 4])
# # # take 20 features  to 10 nodes in hidden layer
w1 = tf.Variable(tf.random_normal([4, 1000],stddev=.5,name='w1'))
# # # add biases for each node
b1 = tf.Variable(tf.zeros([1000]))
# # calculate activations 
hidden_output = tf.nn.softmax(tf.matmul(x, w1) + b1)

w2 = tf.Variable(tf.random_normal([1000, 2],stddev=.5,name='w2'))
b2 = tf.Variable(tf.zeros([2]))

# # placeholder for correct values 
y_ = tf.placeholder("float", [None,2])
# # #implement model. these are predicted ys
y = tf.nn.softmax(tf.matmul(hidden_output, w2) + b2)

In [0]:
loss = tf.reduce_mean(tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_, name='xentropy')))
opt = tf.train.AdamOptimizer(learning_rate=.002)
train_step = opt.minimize(loss, var_list=[w1,b1,w2,b2])

In [0]:
def get_mini_batch(x,y):
	rows=np.random.choice(x.shape[0], 50)
	return x[rows], y[rows]

In [0]:
sess = tf.Session()
# init all vars
init = tf.initialize_all_variables()
sess.run(init)

In [197]:
ntrials = 10000
for i in range(ntrials):
    # get mini batch
    a,b=get_mini_batch(x_train,y_train)
    # run train step, feeding arrays of 100 rows each time
    _, cost =sess.run([train_step,loss], feed_dict={x: a, y_: b})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Leave few out Test accuracy on v2 is {}".format(sess.run(accuracy, feed_dict={x: x_test, y_: y_test})))

Leave few out Test accuracy on v2 is 0.99880951643


In [198]:
saver = tf.train.Saver()
#saver.save(sess, 'twoDigitlClassifier',global_step=1000)
saver.save(sess, './model_final')

'./model_final'

In [199]:
#Testing on new data
df = pd.read_csv("test.csv")

label=df.pop("label")

train_labels = pd.get_dummies(label)
train_labels = train_labels.values
train_labels = np.array(train_labels[:, 0])

y_test = (np.arange(2) == train_labels[:, None])
y_test = np.array(y_test,dtype='float32')
x_test = np.array(df,dtype='float32')


print len(y_test), len(x_test)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



2217 2217


In [200]:
print("Test accuracy on v1 samples is {}".format(sess.run(accuracy, feed_dict={x: x_test, y_: y_test})))

Test accuracy on v1 samples is 0.0
